# Load Unstructure Data from Office files
```bash
pip install unstructured openpyxl python-magic python-pptx
pip install langchain-unstructured
pip install nltk
```

In [29]:
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import textwrap
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
loader = UnstructuredPowerPointLoader("docs\\xLLM.pptx", mode="elements")
docs = loader.load()

In [31]:
ppt_data = {}
for doc in docs:
    page = doc.metadata["page_number"]
    ppt_data[page] = ppt_data.get(page, "") + '\n\n' + doc.page_content

ppt_data

{1: '\n\n1\n\nMLtechniques.com – xLLM, by Vincent Granville\n\nFast, High Performance Multi-LLM System Without Neural Network\n\nxLLM: New Generation of Large Language Models\x0b\x0bVincent Granville, PhD\x0bChief AI Scientist \x0bGenAItechLab.com\x0bvincentg@mltechniques.com\x0b \x0bMay 29, 2024\n\n',
 2: '\n\n2\n\nMLtechniques.com - xLLM, by Vincent Granville\n\nExtreme LLM (xLLM) in a Snapshot\n\nMixture of experts\n\nOne specialized, small sub-LLM per top category\n\nLLM router (multi-agent system) to manage the sub-LLMs\n\nUser selects top category and hyperparameters\n\nEach sub-LLM built with its own taxonomy and knowledge graph\n\nNo neural network, no training\n\nThus, low cost, easy to fine-tune (in milliseconds per sub-LLM)\n\nSelf-tuned based on favorite hyperparameters, and customizable\n\nNo GPU, no latency, better results, local implementation\n\n',
 3: '\n\n3\n\nMLtechniques.com - xLLM, by Vincent Granville\n\nExtreme LLM (xLLM) in a Snapshot\n\nConcise results\n\nMulti

In [32]:
context = ""
for page, content in ppt_data.items():
    context += f"### slide {page}: \n\n{content.strip()}\n\n"

context

'### slide 1: \n\n1\n\nMLtechniques.com – xLLM, by Vincent Granville\n\nFast, High Performance Multi-LLM System Without Neural Network\n\nxLLM: New Generation of Large Language Models\x0b\x0bVincent Granville, PhD\x0bChief AI Scientist \x0bGenAItechLab.com\x0bvincentg@mltechniques.com\x0b \x0bMay 29, 2024\n\n### slide 2: \n\n2\n\nMLtechniques.com - xLLM, by Vincent Granville\n\nExtreme LLM (xLLM) in a Snapshot\n\nMixture of experts\n\nOne specialized, small sub-LLM per top category\n\nLLM router (multi-agent system) to manage the sub-LLMs\n\nUser selects top category and hyperparameters\n\nEach sub-LLM built with its own taxonomy and knowledge graph\n\nNo neural network, no training\n\nThus, low cost, easy to fine-tune (in milliseconds per sub-LLM)\n\nSelf-tuned based on favorite hyperparameters, and customizable\n\nNo GPU, no latency, better results, local implementation\n\n### slide 3: \n\n3\n\nMLtechniques.com - xLLM, by Vincent Granville\n\nExtreme LLM (xLLM) in a Snapshot\n\nConci

In [33]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


base_url = "http://localhost:11434"
model = "llama3.2:3b"

llm = ChatOllama(base_url=base_url,
                 model=model,
                 temperature=0.1,
                 num_predict=256
                 )

system = SystemMessagePromptTemplate.from_template("""
You are a helpful AI assistant who answer user question based on provide context.
""")

prompt = """Answer user question based on the provided context ONLY! if do not know the answer, just sai "I don't know".
### Context:
{context}

### Question:
{question}

### Answer:
"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

qna_chain = template | llm | StrOutputParser()

In [34]:
def ask_llm(context, question):
    return qna_chain.invoke({'context':context, 'question':question}) 

In [44]:
question = """For each PowerPoint slide provided above, write in MarkDown format the key points.
Ensure smooth flow between slides, maintaining a clear and engaging narrative.
"""

answer = ask_llm(content, question)
print(answer)

I don't know how to generate content for PowerPoint slides based on the provided context. However, I can provide you with a general outline of what the key points might be for each slide:

**Slide 1: Introduction**
* Title: "xLLM"
* Subtitle: "A Novel Approach to Large Language Models"
* Key point: Introducing xLLM, a new method for large language models that improves upon existing approaches.

**Slide 2: Problem Statement**
* Title: "Challenges in Large Language Models"
* Bullet points:
	+ Limited contextual understanding
	+ Inefficient use of resources
	+ Difficulty in handling out-of-vocabulary words
* Key point: Highlighting the limitations and challenges of traditional large language models.

**Slide 3: Solution Overview**
* Title: "xLLM: A Novel Approach"
* Bullet points:
	+ Adaptive loss function
	+ Smart crawling and x-embeddings
	+ Fast approximate nearest neighbor search
* Key point: Introducing the key components of xLLM and how they address the challenges mentioned earlier.